<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_nonogram_in_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a nonogram in PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

Let's start by installing `borb`

In [1]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.3 MB 12.2 MB/s 
     |████████████████████████████████| 212 kB 61.1 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 930 kB 75.8 MB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
  Created wheel for qrcode: filename=qrcode-7.3.1-py3-none-any.whl size=40402 sha256=595c57a754cc9280bb00243d06f02526ed65cdb6e7d0802ab272f8fa1e739e60
  Stored in directory: /root/.cache/pip/wheels/93/d7/39/a4111be2cfb8e679938aa671a37888b6afb1f9e7d748e94492
Successfully built qrcode
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency confl

We're going to define the final nonogram as a piece of ASCII art:

In [2]:
ascii_art: str = """
■...........■..
■...........■..
■■■.■■■.■■■.■■■
■.■.■.■.■...■.■
■■■.■■■.■...■■■
"""

Now we need to turn this into a set of horizontal and vertical clues. The following code does just that!

In [3]:
import typing

# trim
while ascii_art[0] == '\n':
  ascii_art = ascii_art[1:]
while ascii_art[-1] == '\n':
  ascii_art = ascii_art[:-1]

# horizontal clues
horizontal_clues: typing.List[typing.List[int]] = []
for row in ascii_art.split('\n'):
  prev_char: str = ''
  prev_count: int = 0
  row_clues: typing.List[int] = []
  for c in row:
    if c == prev_char:
      prev_count += 1
    else:
      if prev_char == '■':
        row_clues.append(prev_count)
      prev_char = c
      prev_count = 1
  if prev_char == '■':
    row_clues.append(prev_count)    
  horizontal_clues.append(row_clues)
number_of_rows: int = len(horizontal_clues)

# vertical clues
number_of_cols: int = int(len(ascii_art) / number_of_rows)
vertical_clues: typing.List[typing.List[int]] = []
for col_index in range(0, number_of_cols):
  col = [ascii_art.split('\n')[i][col_index] for i in range(0, number_of_rows)]
  prev_char: str = ''
  prev_count: int = 0
  col_clues: typing.List[int] = []
  for c in col:
    if c == prev_char:
      prev_count += 1
    else:
      if prev_char == '■':
        col_clues.append(prev_count)
      prev_char = c
      prev_count = 1
  if prev_char == '■':
    col_clues.append(prev_count)
  vertical_clues.append(col_clues)

# padding for horizontal_clues
max_number_of_horizontal_clues: int = max([len(x) for x in horizontal_clues])
for row in horizontal_clues:
  while len(row) < max_number_of_horizontal_clues:
    row.insert(0, None)

# padding for vertical_clues
max_number_of_vertical_clues: int = max([len(x) for x in vertical_clues])
for col in vertical_clues:
  while len(col) < max_number_of_vertical_clues:
    col.insert(0, None)

For this PDF we're going to use a custom `Font`. Let's first download the `ttf` file.

In [4]:
from borb.pdf.canvas.font.simple_font.true_type_font import TrueTypeFont  
from borb.pdf.canvas.font.font import Font

# Download Font
import requests
with open('IndieFlower-Regular.ttf', 'wb') as ffh:
  ffh.write(requests.get("https://github.com/google/fonts/blob/main/ofl/indieflower/IndieFlower-Regular.ttf?raw=true", allow_redirects=True).content)

Now we can create a skeleton document containing our title and explanation blurb:

In [5]:
from borb.pdf.document.document import Document
from borb.pdf.page.page import Page
from borb.pdf.pdf import PDF
from borb.pdf.canvas.layout.page_layout.multi_column_layout import SingleColumnLayout
from borb.pdf.canvas.layout.page_layout.page_layout import PageLayout
from borb.pdf.canvas.layout.text.paragraph import Paragraph
from borb.pdf.canvas.color.color import HexColor

from pathlib import Path  
from decimal import Decimal

# create empty Document
pdf = Document()

# create empty Page
page = Page()

# add Page to Document
pdf.add_page(page)

# create PageLayout
layout: PageLayout = SingleColumnLayout(page)

# add title
layout.add(Paragraph('Nonogram', 
                     font_color=HexColor('#19647E'),
                      font=TrueTypeFont.true_type_font_from_file(Path("IndieFlower-Regular.ttf")),
                      font_size=Decimal(20)))
  
# add explanation
layout.add(Paragraph("""
Nonograms, also known as Hanjie, Paint by Numbers, Picross, Griddlers, and Pic-a-Pix, and by various other names, 
are picture logic puzzles in which cells in a grid must be colored or left blank according to numbers at the side of the grid to reveal a hidden picture. 
In this puzzle type, the numbers are a form of discrete tomography that measures how many unbroken lines of filled-in squares there are in any given row or column. 
For example, a clue of "4 8 3" would mean there are sets of four, eight, and three filled squares, in that order, with at least one blank square between successive sets.
                    """,
                     font_color=HexColor('#28AFB0')))

We're going to represent the nonogram as a `Table`.
The following code builds a `FixedColumnWidthTable` from the clues we defined earlier.

In [6]:
# new imports
from borb.pdf.canvas.layout.table.fixed_column_width_table import FixedColumnWidthTable
from borb.pdf.canvas.layout.table.table import TableCell
from borb.pdf.canvas.layout.layout_element import Alignment

table:FixedColumnWidthTable = FixedColumnWidthTable(number_of_rows=max_number_of_vertical_clues+number_of_rows, 
                                                    number_of_columns=max_number_of_horizontal_clues+number_of_cols)

for i in range(0, max_number_of_vertical_clues):
  for _ in range(0, max_number_of_horizontal_clues):
    table.add(TableCell(Paragraph(" "), border_top=False, border_right=False, border_bottom=False, border_left=False))
  for j in range(0, len(vertical_clues)):
    if vertical_clues[j][i] is None:
      table.add(TableCell(Paragraph(" "), border_top=False, border_right=False, border_bottom=False, border_left=False))
    else:
      table.add(TableCell(Paragraph(str(vertical_clues[j][i]), horizontal_alignment=Alignment.CENTERED), 
                          border_top=False, 
                          border_right=False, 
                          border_bottom=False, 
                          border_left=False))

for i in range(0, len(horizontal_clues)):
  for j in horizontal_clues[i]:
    if j is None:
      table.add(TableCell(Paragraph(" "), border_top=False, border_right=False, border_bottom=False, border_left=False))
    else:
      table.add(TableCell(Paragraph(str(j), horizontal_alignment=Alignment.CENTERED), border_top=False, border_right=False, border_bottom=False, border_left=False))
  for _ in range(0, number_of_cols):
      table.add(Paragraph(" "))

table.set_padding_on_all_cells(Decimal(2), Decimal(2), Decimal(2), Decimal(2))

# add Table
layout.add(table)

Finally, we can store the `PDF`:

In [7]:
# write Document
with open("output_001.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, pdf)